In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score,ConfusionMatrixDisplay,hamming_loss,classification_report

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import utils, callbacks
from tensorflow.keras import utils
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models
from keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
#hello

In [ ]:
train_folder = '/kaggle/input/indian-sign-language/Train'
all_data = []
for folder in os.listdir(train_folder):
    
    label_folder = os.path.join(train_folder, folder)    
    onlyfiles = [{'label':folder,'path':os.path.join(label_folder, f)} for f in os.listdir(label_folder) if os.path.isfile(os.path.join(label_folder, f))]
    #print(onlyfiles)
    all_data += onlyfiles
    #print(all_data)
data_df = pd.DataFrame(all_data)
data_df

In [ ]:
test_folder = '/kaggle/input/indian-sign-language/Test'
all_data = []
for folder in os.listdir(test_folder):
    
    label_folder = os.path.join(test_folder, folder)    
    onlyfiles = [{'label':folder,'path':os.path.join(label_folder, f)} for f in os.listdir(label_folder) if os.path.isfile(os.path.join(label_folder, f))]
    #print(onlyfiles)
    all_data += onlyfiles
    #print(all_data)
test_df = pd.DataFrame(all_data)
test_df

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from scipy.ndimage import gaussian_filter
import numpy as np
#import skimage.util
from PIL import Image, ImageFilter 
from PIL import ImageEnhance
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import random
import cv2

img_width, img_height = 128, 128
batch_size = 128
y_col = 'label'
x_col = 'path'
no_of_classes = len(data_df[y_col].unique())

train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,    
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=data_df, x_col=x_col, y_col=y_col,
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

# def elastic_transform(image, alpha=2, sigma=0.05, random_state=None):
#     if random_state is None:
#         random_state = np.random.RandomState(None)

#     shape = image.shape
#     dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
#     dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
#     dz = np.zeros_like(dx)

#     x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
#     indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

#     distored_image = map_coordinates(image, indices, order=1, mode='reflect')
#     return distored_image.reshape(image.shape)

# def color_jittering(image, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1):
#     jittered_image = Image.fromarray(np.array(image).astype(np.uint8))

#     enhancer = ImageEnhance.Brightness(jittered_image)
#     jittered_image = enhancer.enhance(1 + random.uniform(-brightness, brightness))

#     enhancer = ImageEnhance.Contrast(jittered_image)
#     jittered_image = enhancer.enhance(1 + random.uniform(-contrast, contrast))

#     enhancer = ImageEnhance.Color(jittered_image)
#     jittered_image = enhancer.enhance(1 + random.uniform(-saturation, saturation))

#     hsv_image = jittered_image.convert("HSV")
#     hue_shift = random.uniform(-hue, hue) * 255
#     hsv_image = hsv_image.point(lambda x: (x + hue_shift) % 256)
#     jittered_image = hsv_image.convert("RGB")
#     jittered_image = cv2.cvtColor(np.array(jittered_image), cv2.COLOR_RGB2BGR)

#     return jittered_image


# def gaussian_blur(image, sigma=1):
    
#     return cv2.GaussianBlur(image, (3, 3), 0)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from scipy.ndimage import gaussian_filter
import numpy as np
#import skimage.util
from PIL import Image, ImageFilter 
from PIL import ImageEnhance
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import random
import cv2

img_width, img_height = 128, 128
batch_size = 128
y_col = 'label'
x_col = 'path'
no_of_classes = len(test_df[y_col].unique())

test_datagen = ImageDataGenerator(
    rescale=1/255.0,
        
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df, x_col=x_col, y_col=y_col,
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
comp_recorded_times=[]
comp_recorded_accuracy=[]
comp_recorded_loss=[]


classes = 36
epochs = 10
learning_rate = 0.0001
steps_per_epoch=np.ceil(103282/128) 
adam = Adam(learning_rate=learning_rate)

class TimePerEpochCallback(Callback):
    def on_train_begin(self, logs=None):
        self.times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        self.times.append(epoch_time)
        print(f"Time taken for epoch {epoch + 1}: {epoch_time:.2f} seconds")


In [ ]:
models_to_compare=[]
xception_base = Xception(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
for layer in xception_base.layers:
    layer.trainable = False
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
prediction_layer = layers.Dense(36, activation='softmax')
model = models.Sequential([
    xception_base,
    flatten_layer,
    dense_layer_1,
    dropout_layer_1,
    prediction_layer
])
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
models_to_compare.append(("Xception", model))


from tensorflow.keras.applications import VGG19
vgg19_base = VGG19(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
for layer in vgg19_base.layers:
    layer.trainable = False
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
prediction_layer = layers.Dense(36, activation='softmax')
model = models.Sequential([
    vgg19_base,
    flatten_layer,
    dense_layer_1,
    dropout_layer_1,
    prediction_layer
])
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
models_to_compare.append(("VGG19", model))


from tensorflow.keras.applications import ResNet50
Resnet50V1_base = ResNet50(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
for layer in Resnet50V1_base.layers:
    layer.trainable = False
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
prediction_layer = layers.Dense(36, activation='softmax')
model = models.Sequential([
    Resnet50V1_base,
    flatten_layer,
    dense_layer_1,
    dropout_layer_1,
    prediction_layer
])
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
models_to_compare.append(("ResNet50", model))


from tensorflow.keras.applications import EfficientNetB7
EfficientNetB7_base = EfficientNetB7(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
for layer in EfficientNetB7_base.layers:
    layer.trainable = False
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
prediction_layer = layers.Dense(36, activation='softmax')
model = models.Sequential([
    EfficientNetB7_base,
    flatten_layer,
    dense_layer_1,
    dropout_layer_1,
    prediction_layer
])
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
models_to_compare.append(("EfficientNet", model))

In [ ]:
def train_model_with_time(model, data_generator, num_epochs):
    epochs = 10
    recorded_times = []
    recorded_accuracy = []
    recorded_loss = []
    for epoch in range(epochs):
        start_time = time.time()
        print("epoch: ",epoch)
        history = model.fit(data_generator, epochs=1, verbose=0)
        end_time = time.time()

        # Record metrics
        epoch_time = end_time - start_time
        epoch_accuracy = history.history['accuracy'][0]
        epoch_loss = history.history['loss'][0]

        recorded_times.append(epoch_time)
        recorded_accuracy.append(epoch_accuracy)
        recorded_loss.append(epoch_loss)

        print(f"Epoch {epoch + 1}/{epochs} - Time: {epoch_time:.2f}s - Accuracy: {epoch_accuracy:.4f} - Loss: {epoch_loss:.4f}")

    return recorded_loss, recorded_accuracy, recorded_times

In [ ]:
results=[]
for model_name, model in models_to_compare:
    print(f"Training {model_name}...")
    loss, accuracy, epoch_times = train_model_with_time(model, train_generator, epochs)
    results.append((model_name, loss, accuracy, epoch_times))

In [ ]:
import matplotlib.pyplot as plt

# Compare the loss of each epoch for each model
for model_name, loss, _, _ in results:
    plt.plot(range(1, epochs + 1), loss, label=model_name)

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Comparison for Different Models')
plt.legend()
plt.show()

# Compare the accuracy of each epoch for each model
for model_name, _, accuracy, _ in results:
    plt.plot(range(1, epochs + 1), accuracy, label=model_name)

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison for Different Models')
plt.legend()
plt.show()

# Compare the epoch times for each model
for model_name, _, _, epoch_times in results:
    plt.plot(range(1, epochs + 1), epoch_times, label=model_name)

plt.xlabel('Epochs')
plt.ylabel('Time per Epoch (seconds)')
plt.title('Epoch Times Comparison for Different Models')
plt.legend()
plt.show()

In [ ]:
history_filename="results.pkl"
with open("results.pkl","wb") as file:
    pickle.dump(results, file)
os.rename(history_filename, '/kaggle/working/' + history_filename)


In [ ]:
Xception = tf.keras.models.load_model('/kaggle/working/Xception.h5')
VGG19 = tf.keras.models.load_model('/kaggle/working/VGG19.h5')
ResNet50 = tf.keras.models.load_model('/kaggle/working/ResNet50.h5')
#EfficientNet = tf.keras.models.load_model('/kaggle/working/EfficientNet.h5')

In [ ]:
# Make predictions using the models
y_pred1 = Xception.predict(test_generator)
y_pred2 = VGG19.predict(test_generator)
y_pred3 = ResNet50.predict(test_generator)

# Get predicted classes
y_pred_classes1 = np.argmax(y_pred1, axis=1)
y_pred_classes2 = np.argmax(y_pred2, axis=1)
y_pred_classes3 = np.argmax(y_pred3, axis=1)


In [ ]:
acc_test={}
acc_test["Xception"]=accuracy_score(true_labels,y_pred_classes1)
acc_test["VGG19"]=accuracy_score(true_labels,y_pred_classes2)
acc_test["ResNet50"]=accuracy_score(true_labels,y_pred_classes3)
for x in acc_test:
    acc_test[x]*=100
    print(x," : ",acc_test[x])
categories=acc_test.keys()
values=acc_test.values()
plt.bar(categories, values)
    

In [ ]:
true_labels= test_generator.classes
print(np.unique(true_labels))

In [ ]:
hamming_loss_dict={}
hamming_loss_dict["Xception"]=hamming_loss(true_labels,y_pred_classes1)
hamming_loss_dict["VGG19"]=hamming_loss(true_labels,y_pred_classes2)
hamming_loss_dict["ResNet50"]=hamming_loss(true_labels,y_pred_classes3)
for  x in hamming_loss_dict:
    print(x," : ", hamming_loss_dict[x]*)

In [ ]:
confusion_mat = {}
confusion_mat["Xception"]=confusion_matrix(true_labels, y_pred_classes1)
confusion_mat["VGG19"]=confusion_matrix(true_labels, y_pred_classes2)
confusion_mat["ResNet50"]=confusion_matrix(true_labels, y_pred_classes3)
for x in confusion_mat:
    print(x)
    cmd=(ConfusionMatrixDisplay(confusion_mat[x]))
    fig, ax = plt.subplots(figsize=(15, 15))
    cmd.plot(ax=ax)

In [ ]:
class_report = {}
class_report["Xception"]=classification_report(true_labels, y_pred_classes1)
class_report["VGG19"]=classification_report(true_labels, y_pred_classes2)
class_report["ResNet50"]=classification_report(true_labels, y_pred_classes3)
for x in class_report:
    print(x)
    print(class_report[x])

In [ ]:
import numpy as np

def custom_generator(train_generator):
    for batch_images, batch_labels in train_generator:
        batch_size = len(batch_images)
        augmented_images = []
        augmented_labels = []

        for i in range(batch_size):
            image = batch_images[i]
            label = batch_labels[i]
            image1=np.expand_dims(image, axis=0)
            yield image1,np.tile(label, (image1.shape[0], 1))
            
#             # Apply elastic transformations
#             transformed_image = elastic_transform(image)
#             transformed_image = np.expand_dims(transformed_image, axis=0)
#             yield transformed_image, np.tile(label, (transformed_image.shape[0], 1))
            
#             # Apply Gaussian noise
#             noisy_image = skimage.util.random_noise(image)
#             noisy_image = np.expand_dims(noisy_image, axis=0)            
#             yield noisy_image, np.tile(label, (noisy_image.shape[0], 1))

#             # Apply color jittering
#             jittered_image = color_jittering(image)
#             jittered_image = np.expand_dims(jittered_image, axis=0)                        
#             yield jittered_image, np.tile(label, (jittered_image.shape[0], 1))            

#             # Apply Gaussian blur
#             blurred_image = gaussian_blur(image)
#             blurred_image = np.expand_dims(blurred_image, axis=0)                                    
#             yield blurred_image, np.tile(label, (blurred_image.shape[0], 1))

custom_train_generator = custom_generator(train_generator)



In [ ]:
xception_base = Xception(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
for layer in xception_base.layers:
    layer.trainable = False
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
prediction_layer = layers.Dense(36, activation='softmax')


model = models.Sequential([
    xception_base,
    flatten_layer,
    dense_layer_1,
    dropout_layer_1,
    prediction_layer
])
model.summary()


In [ ]:
history=model.fit(
    train_generator,
    epochs=epochs
)

In [ ]:
import h5py
import os
import pickle

In [ ]:
model_filename = 'model_Xception_vanilla.h5'
model.save(model_filename)
os.rename(model_filename, '/kaggle/working/' + model_filename)
history_filename="historyXception_vanilla.pkl"
with open('historyXception_vanilla.pkl', 'wb') as file:
    pickle.dump(history.history, file)
os.rename(history_filename, '/kaggle/working/' + history_filename)



In [ ]:

model.summary()


In [ ]:
classes = 36
epochs = 10
learning_rate = 0.0001
steps_per_epoch=np.ceil(103282/128) 
adam = Adam(lr=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
history=model.fit(
    train_generator,
    epochs=epochs
)

In [ ]:
model_filename = 'model_vgg19_vanilla.h5'
model.save(model_filename)
os.rename(model_filename, '/kaggle/working/' + model_filename)
history_filename="history_vgg19_vanilla.pkl"
with open('history_vgg19_vanilla.pkl', 'wb') as file:
    pickle.dump(history.history, file)
os.rename(history_filename, '/kaggle/working/' + history_filename)



In [ ]:
from tensorflow.keras.applications import ResNet50
Resnet50V1_base = ResNet50(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
for layer in Resnet50V1_base.layers:
    layer.trainable = False
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
prediction_layer = layers.Dense(36, activation='softmax')


model = models.Sequential([
    Resnet50V1_base,
    flatten_layer,
    dense_layer_1,
    dropout_layer_1,
    prediction_layer
])
model.summary()


In [ ]:
classes = 36
epochs = 10
learning_rate = 0.0001
steps_per_epoch=np.ceil(103282/128) 
adam = Adam(lr=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
history=model.fit(
    train_generator,
    epochs=epochs
)

In [ ]:
model_filename = 'model_resnet_vanilla.h5'
model.save(model_filename)
os.rename(model_filename, '/kaggle/working/' + model_filename)
history_filename="history_resnet_vanilla.pkl"
with open('history_resnet_vanilla.pkl', 'wb') as file:
    pickle.dump(history.history, file)
os.rename(history_filename, '/kaggle/working/' + history_filename)

In [ ]:
from tensorflow.keras.applications import EfficientNetB7
EfficientNetB7_base = EfficientNetB7(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
for layer in EfficientNetB7_base.layers:
    layer.trainable = False
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
prediction_layer = layers.Dense(36, activation='softmax')


model = models.Sequential([
    EfficientNetB7_base,
    flatten_layer,
    dense_layer_1,
    dropout_layer_1,
    prediction_layer
])
model.summary()


In [ ]:
classes = 36
epochs = 10
learning_rate = 0.0001
steps_per_epoch=np.ceil(103282/128) 
adam = Adam(lr=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
history=model.fit(
    train_generator,
    epochs=epochs
)

In [ ]:
# def elastic_transform(image, alpha=2, sigma=0.05, random_state=None):
#     """Elastic deformation of images as described in [Simard2003]_.
#     .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
#        Convolutional Neural Networks applied to Visual Document Analysis", in
#        Proc. of the International Conference on Document Analysis and
#        Recognition, 2003.
#     """
#     if random_state is None:
#         random_state = np.random.RandomState(None)

#     shape = image.shape
#     dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
#     dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
#     dz = np.zeros_like(dx)

#     x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
#     print(x.shape)
#     indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

#     distored_image = map_coordinates(image, indices, order=1, mode='reflect')
#     return distored_image.reshape(image.shape)

# def add_gaussian_noise(image, mean=0, std=1):
#     noisy_image = np.array(image)
#     noise = np.random.normal(mean, std, size=noisy_image.shape).astype(np.uint8)
#     noisy_image = cv2.add(image, noise)
#     return Image.fromarray(noisy_image)

# def color_jittering(image, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1):
#     jittered_image = Image.fromarray(np.array(image).astype(np.uint8))

#     enhancer = ImageEnhance.Brightness(jittered_image)
#     jittered_image = enhancer.enhance(1 + random.uniform(-brightness, brightness))

#     enhancer = ImageEnhance.Contrast(jittered_image)
#     jittered_image = enhancer.enhance(1 + random.uniform(-contrast, contrast))

#     enhancer = ImageEnhance.Color(jittered_image)
#     jittered_image = enhancer.enhance(1 + random.uniform(-saturation, saturation))

#     hsv_image = jittered_image.convert("HSV")
#     hue_shift = random.uniform(-hue, hue) * 255
#     hsv_image = hsv_image.point(lambda x: (x + hue_shift) % 256)
#     jittered_image = hsv_image.convert("RGB")
    
#     jittered_image = cv2.cvtColor(np.array(jittered_image), cv2.COLOR_RGB2BGR)

#     return jittered_image


# def gaussian_blur(image, sigma=1):
    
#     return cv2.GaussianBlur(image, (3, 3), 0)


In [ ]:
images=[]
image=cv2.imread("/kaggle/input/indian-sign-language/Test/0/0_1.jpg")
# images.append(skimage.util.random_noise(image))
# images.append(gaussian_blur(image))
# images.append(color_jittering(image))
# images.append(elastic_transform(image))
fig, axes = plt.subplots(2, 2)
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])


In [ ]:
    print(images[2].shape)

# Example usage
image = Image.open("image.jpg")  # Replace "image.jpg" with the actual image path
noisy_image = add_gaussian_noise(image, mean=0, std=10)
jittered_image = color_jittering(image, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
transformed_image = elastic_transform(image, alpha=200, sigma=10)
blurred_image = gaussian_blur(image, sigma=1)
rotated_image = rotate_image(image, angle=30)
adjusted_image = adjust_brightness_contrast(image, brightness=0.2, contrast=0.2)